In [93]:
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import models,layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder

In [94]:
def getDataSize(path):
  image_count = {}

  for x in os.listdir(path):
    image_count[x] = len(os.listdir(os.path.join(path, x)))

  #get number of images present for the given path 
  return image_count

In [95]:
train_path = 'dataset/train'
test_path = 'dataset/test'
valid_path = 'dataset/valid'

In [96]:
batch_size = 16

In [97]:
def generateImageRGB(path):
  generate_data = ImageDataGenerator(rescale=1.0/255.0,
                                horizontal_flip=True,
                                fill_mode='nearest', 
                                zoom_range=0.2,
                                shear_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                rotation_range=30)

  return generate_data.flow_from_directory(path, batch_size=batch_size, target_size=(224, 224), class_mode='categorical')

In [98]:
train_data = generateImageRGB(train_path)
test_data = generateImageRGB(test_path)
valid_data = generateImageRGB(valid_path)

Found 743 images belonging to 4 classes.
Found 57 images belonging to 3 classes.
Found 105 images belonging to 4 classes.


In [99]:
num_classes = len(train_data.class_indices)

In [108]:
import os
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Create input tensor
inputs = Input(shape=(224, 224, 3))  # MobileNetV2 expects input images with shape (224, 224, 3)

# Create MobileNetV2 model with pre-trained weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of MobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)

# Modify the last dense layer to have the same number of units as the number of classes
preds = Dense(num_classes, activation='softmax')(x)

# Combine base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=preds)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

C:\Users\ketul\AppData\Local\Temp\ipykernel_4436\4149486156.py:12: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)


Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_12[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 4,160,708 (15.87 MB)

 Trainable params: 1,902,724 (7.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [109]:
# Train the model
history = model.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=20,
    validation_data=valid_data,
    validation_steps=len(valid_data)
)

Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 36s 610ms/step - accuracy: 0.3879 - loss: 1.3599 - val_accuracy: 0.6095 - val_loss: 0.8575
Epoch 2/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20


c:\Users\ketul\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 471ms/step - accuracy: 0.5614 - loss: 0.8732 - val_accuracy: 0.6000 - val_loss: 0.8621
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 419ms/step - accuracy: 0.6530 - loss: 0.7755 - val_accuracy: 0.5333 - val_loss: 0.9229
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 414ms/step - accuracy: 0.6596 - loss: 0.7385 - val_accuracy: 0.6000 - val_loss: 0.8617
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 414ms/step - accuracy: 0.7586 - loss: 0.6043 - val_accuracy: 0.6571 - val_loss: 0.7688
Epoch 10/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy

In [107]:
# Get the maximum validation accuracy achieved during training
max_val_accuracy = max(history.history['val_accuracy'])

# Print the maximum validation accuracy
print("Best Validation Accuracy:", max_val_accuracy)


Best Validation Accuracy: 0.7428571581840515


In [103]:
test_image_path = 'dataset/test/normal/11 (2).png'

In [104]:
from keras.preprocessing.image import load_img, img_to_array

# Define the target image size
image_size = (224, 224)

# Create an ImageDataGenerator instance with resizing
datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values

# Load a single image for testing

test_img = load_img(test_image_path, target_size=image_size)
test_img_array = img_to_array(test_img)
test_img_array = np.expand_dims(test_img_array, axis=0)

# Preprocess the single image using the datagen
prep_img = datagen.standardize(test_img_array)

# Now `preprocessed_img` is ready to be fed to your trained model for prediction


In [105]:

# Now `preprocessed_img` is ready to be fed to your trained model for prediction
output_probabilities = model.predict(prep_img)

# Get the predicted class index
predicted_class_index = np.argmax(output_probabilities)

print("Predicted Class Index:", predicted_class_index)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class Index: 2


In [106]:
from keras.models import load_model

# Save the entire model to a file
model.save('best_model1.keras')